# Using HathiMeta

In [1]:
%load_ext autoreload
%autoreload 2
from compare_tools.utils import HathiMeta

# Load config and print keys
import yaml
with open('../local.yaml') as f:
    config = yaml.safe_load(f)
config.keys()

dict_keys(['rsync_root', 'glove_data_path', 'srp_data_path', 'meta_path'])

## Connecting to the DB

If no DB path is specified, a temporary in-memory database is created.

In [6]:
meta = HathiMeta('test.db')

### First time: Building DB

If you haven't yet ingested the [Hathifiles](https://www.hathitrust.org/hathifiles) or have a transient in-memory db, you'll need to run `create_db`:

In [3]:
meta.create_db(config['meta_path'])

0, 1, 2, 3, 4, 5, 

## Access

In [9]:
len(meta)

143864

In [8]:
meta['uiuo.ark:/13960/t6g20tp0c']

htid                                               uiuo.ark:/13960/t6g20tp0c
access                                                                 allow
rights                                                                    pd
ht_bib_key                                                          11639242
description                                                             v.13
source                                                                   UIU
source_bib_num                                                       2420951
oclc_num                                                            49472649
isbn                                                                    None
issn                                                                    None
lccn                                                                    None
title                      Standard shop efficiency schedules, by Henry W...
imprint                                               Crane & company, 1910-

In [10]:
meta.default_fields = ['title', 'author', 'description']
meta['uiuo.ark:/13960/t6g20tp0c']

title          Standard shop efficiency schedules, by Henry W...
author                              Jacobs, Henry William, 1874-
description                                                 v.13
Name: 0, dtype: object